In [1]:
#pip install azure-cognitiveservices-vision-customvision


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#pip install mediapipe opencv-python


  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
    --------------------------------------- 0.8/51.0 MB 4.2 MB/s eta 0:00:12
    --------------------------------------- 1.0/51.0 MB 2.8 MB/s eta 0:00:18
   - -------------------------------------- 2.4/51.0 MB 3.7 MB/s eta 0:00:14
   --- ------------------------------------ 3.9/51.0 MB 4.9 MB/s eta 0:00:10
   ---- ----------------------------------- 5.5/51.0 MB 5.4 MB/s eta 0:00:09
   ----- ---------------------------------- 7.1/51.0 MB 5.8 MB/s eta 0:00:08
   ------ --------------------------------- 8.4/51.0 MB 6.0 MB/s eta 0:00:08
   ------- -------------------------------- 9.7/51.0 MB 6.0 MB/s eta 0:00:07
   ------- -------------------------------- 9.7/51.0 MB 6.0 MB/s eta 0:00:07
   ------- -------------------------------- 9.7/51.0 MB 6.0 MB/s eta 0:00:07
   ------- -----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


모델 앤드포인트 바꾼 후 바뀐 태그들 맞추기기

## 동영상 프레임 나눠서 이미지 정상 비정상상

In [3]:
import cv2
import os
from collections import Counter
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# Custom Vision 모델 설정
project_settings = {
    "레그레이즈": {
        "ENDPOINT": "https://5b001team1customvision-prediction.cognitiveservices.azure.com",
        "PREDICTION_KEY": "2QpZbyO6aXNggNRFOFDalFoMZ0wlH04rIq3vu84rUAVfYLiYtkXBJQQJ99BAACYeBjFXJ3w3AAAIACOGIbb2",
        "PROJECT_ID": "4799206e-b5be-41a2-9f1e-507aa06e9a89",
        "PUBLISHED_NAME": "LEG_RAISE"
    },
    "푸쉬업": {
        "ENDPOINT": "https://5b001team1customvision-prediction.cognitiveservices.azure.com",
        "PREDICTION_KEY": "2QpZbyO6aXNggNRFOFDalFoMZ0wlH04rIq3vu84rUAVfYLiYtkXBJQQJ99BAACYeBjFXJ3w3AAAIACOGIbb2",
        "PROJECT_ID": "7a47753c-c11b-44af-8ca1-6dbace177a71",
        "PUBLISHED_NAME": "PUSHUP_MAIN"
    }
}

# 비정상 태그 정의
incorrect_tags = {
    "레그레이즈": [
        "incorrect_pose_고개 숙임 여부",
        "incorrect_pose_이완 시 다리 긴장 유지",
        "incorrect_pose_허리와 지면 고정"
    ],
    "푸쉬업": [
        "incorrect_pose_손의 가슴위치 중앙여부",
        "incorrect_pose_이완시 팔꿈치 90도_가슴의 부족한 이동",
        "incorrect_pose_척추의 중립"
    ]
}

def classify_image(file_path, pose_type):
    """ Custom Vision 모델로 이미지를 분류합니다. """
    settings = project_settings[pose_type]
    credentials = ApiKeyCredentials(in_headers={"Prediction-key": settings["PREDICTION_KEY"]})
    predictor = CustomVisionPredictionClient(settings["ENDPOINT"], credentials)

    with open(file_path, "rb") as image_contents:
        results = predictor.classify_image(settings["PROJECT_ID"], settings["PUBLISHED_NAME"], image_contents.read())
    
    top_prediction = results.predictions[0]
    return {"tag": top_prediction.tag_name, "probability": top_prediction.probability}

def extract_frames(video_path, output_folder, max_frames=10):
    """ 동영상에서 프레임을 추출하여 이미지로 저장합니다. """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(frame_count // max_frames, 1)  # 최대 10개 프레임 추출

    extracted_files = []
    frame_idx = 0
    saved_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or saved_count >= max_frames:
            break
        if frame_idx % interval == 0:
            file_name = os.path.join(output_folder, f"frame_{saved_count}.jpg")
            cv2.imwrite(file_name, frame)
            extracted_files.append(file_name)
            saved_count += 1
        frame_idx += 1
    
    cap.release()
    return extracted_files

def analyze_video(video_path, pose_type):
    """ 동영상의 각 프레임을 분석하여 비정상 자세를 판별합니다. """
    frame_folder = "extracted_frames"
    frames = extract_frames(video_path, frame_folder)

    abnormal_count = 0
    abnormal_tags = []

    for frame in frames:
        result = classify_image(frame, pose_type)
        tag = result["tag"]
        probability = result["probability"]

        # 비정상 태그인 경우 기록
        if tag in incorrect_tags[pose_type]:
            abnormal_count += 1
            abnormal_tags.append(tag)

    # 가장 많이 등장한 비정상 태그 찾기
    most_common_abnormal_tag = None
    if abnormal_tags:
        most_common_abnormal_tag = Counter(abnormal_tags).most_common(1)[0][0]

    # 비정상 판단 기준 (10개 중 8개 이상 비정상이면 비정상)
    if abnormal_count >= 8:
        return f"비정상 자세 ({most_common_abnormal_tag})"
    else:
        return "정상 자세"

if __name__ == "__main__":
    pose_type = input("운동 종류를 입력하세요 (레그레이즈/푸쉬업): ").strip()
    video_path = "custom_vision_test/푸쉬업.mp4"  # 분석할 영상 경로

    try:
        result = analyze_video(video_path, pose_type)
        print(f"운동 종류: {pose_type}")
        print(result)

    except FileNotFoundError as e:
        print("파일을 찾을 수 없습니다:", e)
    except ValueError as e:
        print("잘못된 입력:", e)
    except Exception as e:
        print(f"오류 발생: {e}")


운동 종류: 푸쉬업
정상 자세


# 프레임 나뉘는지 테스트


In [5]:
import cv2
import os

def extract_and_display_frames(video_path, output_folder="extracted_frames", max_frames=10):
    """
    동영상에서 프레임을 추출하여 저장하고 화면에 표시합니다.
    
    :param video_path: 분석할 영상 파일 경로 (절대 경로 사용)
    :param output_folder: 추출한 프레임을 저장할 폴더
    :param max_frames: 저장할 최대 프레임 수 (기본 10개)
    """
    # 프레임 저장 폴더 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 영상 불러오기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"영상 파일을 열 수 없습니다: {video_path}")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 총 프레임 수 확인
    interval = max(frame_count // max_frames, 1)  # 프레임 추출 간격 설정

    print(f"총 프레임 수: {frame_count}")
    print(f"프레임 추출 간격: {interval} 프레임마다 1개씩 저장")
    
    frame_idx = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()  # 프레임 읽기
        if not ret or saved_count >= max_frames:
            break  # 더 이상 읽을 프레임이 없거나 최대 프레임 개수에 도달하면 종료
        
        if frame_idx % interval == 0:
            file_name = os.path.join(output_folder, f"frame_{saved_count}.jpg")
            cv2.imwrite(file_name, frame)  # 프레임 저장
            print(f"프레임 저장: {file_name}")

            # 프레임 화면에 표시
            cv2.imshow(f"Frame {saved_count}", frame)
            key = cv2.waitKey(500)  # 0.5초 동안 프레임 표시
            cv2.destroyAllWindows()

            saved_count += 1
        
        frame_idx += 1

    cap.release()
    print("프레임 추출 완료.")
    print(f"총 {saved_count}개의 프레임이 저장되었습니다.")

if __name__ == "__main__":
    # 절대 경로 설정 (주어진 프로젝트 경로와 영상 파일 경로)
    project_path = r"C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test"
    video_path = os.path.join(project_path, "pushup.mp4")

    # 파일 존재 여부 확인
    if not os.path.exists(video_path):
        print(f"파일이 존재하지 않습니다: {video_path}")
    else:
        print(f"파일이 존재합니다: {video_path}")

    # 프레임 추출 및 표시
    extract_and_display_frames(video_path)


파일이 존재합니다: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\pushup.mp4
총 프레임 수: 280
프레임 추출 간격: 28 프레임마다 1개씩 저장
프레임 저장: extracted_frames\frame_0.jpg
프레임 저장: extracted_frames\frame_1.jpg
프레임 저장: extracted_frames\frame_2.jpg
프레임 저장: extracted_frames\frame_3.jpg
프레임 저장: extracted_frames\frame_4.jpg
프레임 저장: extracted_frames\frame_5.jpg
프레임 저장: extracted_frames\frame_6.jpg
프레임 저장: extracted_frames\frame_7.jpg
프레임 저장: extracted_frames\frame_8.jpg
프레임 저장: extracted_frames\frame_9.jpg
프레임 추출 완료.
총 10개의 프레임이 저장되었습니다.


테스트까지 완료된 코드
+ 기존 남아있는 프레임 사진 삭제 + 분석 후 프레임 사진 삭제제

In [8]:
import cv2
import os
from collections import Counter
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# Custom Vision 모델 설정
project_settings = {
    "레그레이즈": {
        "ENDPOINT": "https://5b001team1customvision-prediction.cognitiveservices.azure.com",
        "PREDICTION_KEY": "2QpZbyO6aXNggNRFOFDalFoMZ0wlH04rIq3vu84rUAVfYLiYtkXBJQQJ99BAACYeBjFXJ3w3AAAIACOGIbb2",
        "PROJECT_ID": "4799206e-b5be-41a2-9f1e-507aa06e9a89",
        "PUBLISHED_NAME": "LEG_RAISE"
    },
    "푸쉬업": {
        "ENDPOINT": "https://5b001team1customvision-prediction.cognitiveservices.azure.com",
        "PREDICTION_KEY": "2QpZbyO6aXNggNRFOFDalFoMZ0wlH04rIq3vu84rUAVfYLiYtkXBJQQJ99BAACYeBjFXJ3w3AAAIACOGIbb2",
        "PROJECT_ID": "7a47753c-c11b-44af-8ca1-6dbace177a71",
        "PUBLISHED_NAME": "PUSHUP_MAIN"
    }
}

# 비정상 태그 정의
incorrect_tags = {
    "레그레이즈": [
        "incorrect_pose_고개 숙임 여부",
        "incorrect_pose_이완 시 다리 긴장 유지",
        "incorrect_pose_허리와 지면 고정"
    ],
    "푸쉬업": [
        "incorrect_pose_손의 가슴위치 중앙여부",
        "incorrect_pose_이완시 팔꿈치 90도_가슴의 부족한 이동",
        "incorrect_pose_척추의 중립"
    ]
}

def classify_image(file_path, pose_type):
    """ Custom Vision 모델로 이미지를 분류합니다. """
    print(f"분석할 이미지 경로: {file_path}")  # 디버깅을 위해 경로 출력

    settings = project_settings[pose_type]
    credentials = ApiKeyCredentials(in_headers={"Prediction-key": settings["PREDICTION_KEY"]})
    predictor = CustomVisionPredictionClient(settings["ENDPOINT"], credentials)

    with open(file_path, "rb") as image_contents:
        results = predictor.classify_image(settings["PROJECT_ID"], settings["PUBLISHED_NAME"], image_contents.read())
    
    top_prediction = results.predictions[0]
    return {"tag": top_prediction.tag_name, "probability": top_prediction.probability}

def extract_frames(video_path, output_folder, max_frames=10):
    """ 동영상에서 프레임을 추출하기 전에 기존 프레임 삭제 후 새로 저장합니다. """
    # 기존 프레임 삭제
    if os.path.exists(output_folder):
        for file in os.listdir(output_folder):
            file_path = os.path.join(output_folder, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
    else:
        os.makedirs(output_folder)

    # 영상 열기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"영상 파일을 열 수 없습니다: {video_path}")
        return []
    else:
        print(f"영상 파일 열기 성공: {video_path}")
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"총 프레임 수: {frame_count}")
    interval = max(frame_count // max_frames, 1)  # 최대 10개 프레임 추출

    extracted_files = []
    frame_idx = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or saved_count >= max_frames:
            break
        if frame_idx % interval == 0:
            file_name = os.path.join(output_folder, f"frame_{saved_count}.jpg")
            cv2.imwrite(file_name, frame)
            extracted_files.append(file_name)
            print(f"프레임 저장: {file_name}")
            saved_count += 1
        frame_idx += 1

    cap.release()
    print(f"총 {saved_count}개의 프레임이 저장되었습니다.")
    return extracted_files

def analyze_video(video_path, pose_type):
    """ 동영상의 각 프레임을 분석하여 비정상 자세를 판별합니다. """
    frame_folder = os.path.join(os.path.dirname(video_path), "extracted_frames")
    frames = extract_frames(video_path, frame_folder)

    if not frames:
        return "프레임을 추출하지 못했습니다."

    abnormal_count = 0
    abnormal_tags = []

    for frame in frames:
        result = classify_image(frame, pose_type)
        tag = result["tag"]
        probability = result["probability"]

        # 비정상 태그인 경우 기록
        if tag in incorrect_tags[pose_type]:
            abnormal_count += 1
            abnormal_tags.append(tag)

    # 가장 많이 등장한 비정상 태그 찾기
    most_common_abnormal_tag = None
    if abnormal_tags:
        most_common_abnormal_tag = Counter(abnormal_tags).most_common(1)[0][0]

    # 분석 후 프레임 자동 삭제
    for frame in frames:
        if os.path.exists(frame):
            os.remove(frame)
    if os.path.exists(frame_folder):
        os.rmdir(frame_folder)

    # 비정상 판단 기준 (10개 중 8개 이상 비정상이면 비정상)
    if abnormal_count >= 8:
        return f"비정상 자세 ({most_common_abnormal_tag})"
    else:
        return "정상 자세"

if __name__ == "__main__":
    # 프로젝트 절대 경로 설정
    project_path = r"C:\projects\jupyterProjects\custom_vision_test_Final_project"
    video_path = os.path.join(project_path, "pushup.mp4")

    # 운동 종류 입력 받기
    pose_type = input("운동 종류를 입력하세요 (레그레이즈/푸쉬업): ").strip()

    # 파일 확인
    if not os.path.exists(video_path):
        print(f"파일이 존재하지 않습니다: {video_path}")
    else:
        try:
            result = analyze_video(video_path, pose_type)
            print(f"운동 종류: {pose_type}")
            print(result)
        except FileNotFoundError as e:
            print("파일을 찾을 수 없습니다:", e)
        except ValueError as e:
            print("잘못된 입력:", e)
        except Exception as e:
            print(f"오류 발생: {e}")


영상 파일 열기 성공: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\pushup.mp4
총 프레임 수: 280
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_0.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_1.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_2.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_3.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_4.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_5.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-School-3차 프로젝트\custom_vision_test\extracted_frames\frame_6.jpg
프레임 저장: C:\Users\USER\Desktop\MS-AI-School 자료 모음\MS-AI-S